In [ ]:
import os 
from sklearn.preprocessing import RobustScaler
import pandas as pd

DIR = os.path.join(os.path.dirname(os.getcwd()), "img")
DATA_DIR = os.path.join(os.path.dirname(os.getcwd()), "data")
DPI = 300

raw_signals = pd.read_pickle(f"{DATA_DIR}/raw_signals.pkl")
crsp = pd.read_pickle(f"{DATA_DIR}/crsp.pkl")

In [14]:
obs_thresh = 240

In [ ]:
S = raw_signals["mom_12_1"].unstack()
last_valid_obs = S.iloc[-1].notnull()
enough_obs = S.count() > obs_thresh
is_valid = enough_obs & last_valid_obs  # Time series to be used to estimate the parameters
S = S.loc[:, is_valid]  # Filtered subset

permno,10145.0,10516.0,10866.0,10874.0,10890.0,11308.0,11404.0,11674.0,11850.0,12036.0,...,89916.0,89922.0,89935.0,89941.0,89948.0,89952.0,89960.0,89964.0,89971.0,89973.0
1963-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1963-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,0.053542,-0.272588,0.257137,-0.005133,0.117391,0.092710,0.070861,0.142496,0.042352,0.136698,...,0.108297,0.120961,-0.159727,0.272588,0.023521,-0.850728,0.358379,0.166088,0.072933,0.820413
2024-09-30,0.090882,-0.231164,0.339999,-0.028840,0.423620,0.233547,0.158813,0.209797,-0.011725,0.225043,...,0.166715,0.134550,-0.617715,0.681031,-0.023887,-1.192554,0.404594,0.221335,0.151887,0.687187
2024-10-31,0.093855,-0.198603,0.216748,0.159884,0.668254,0.223405,0.158403,0.275090,0.088450,0.206635,...,0.157713,0.316539,-0.673837,0.492094,0.039670,-0.755688,0.600530,0.253920,0.275657,0.575858
2024-11-30,0.016735,-0.313255,-0.057018,-0.022226,0.319539,0.085981,0.099817,0.164808,0.105885,0.205021,...,0.197729,0.267807,-0.735272,0.475121,-0.018300,-0.716470,0.267047,0.256757,0.132404,0.308182


In [ ]:
S.xs("2024-12-31")

permno
10026.0    0.005539
10028.0    0.274403
10032.0    0.372807
10044.0   -0.583589
10066.0    0.189935
             ...   
93397.0    0.261936
93426.0   -0.443939
93429.0    0.154830
93434.0   -0.689228
93436.0    0.281452
Name: mom_12_1, Length: 3656, dtype: float64